In [ ]:
import os
import numpy as np
import pandas as pd

x_file_path = './CellTOSG/brain_sc_output/processed_data/brain/alzheimer\'s_disease/alzheimer\'s_disease_X_partition_0.npy'
y_file_path = './CellTOSG/brain_sc_output/processed_data/brain/alzheimer\'s_disease/alzheimer\'s_disease_Y_partition_0.npy'

x = np.load(x_file_path)
y = np.load(y_file_path)
print(x.shape, y.shape)

(291, 483817) (291,)


In [ ]:
edge_index_path = './CellTOSG/brain_sc_output/edge_index.npy'
x_desc_path = './CellTOSG/brain_sc_output/X_descriptions.npy'

edge_index = np.load(edge_index_path)
x_desc = np.load(x_desc_path, allow_pickle=True)
print(edge_index.shape, x_desc.shape)
print(edge_index)

(2, 33349084) (483817, 1)
[[     0      1      2 ... 483091 483091 483091]
 [278326 278327 278328 ... 386373 398848 437378]]


In [123]:
x_desc[278326]

array(['GTP-binding protein involved in protein trafficking; may modulate vesicle budding and uncoating within the Golgi apparatus. {ECO:0000250|UniProtKB:P84084}.;  (Microbial infection) Functions as an allosteric activator of the cholera toxin catalytic subunit, an ADP-ribosyltransferase. {ECO:0000305}.'],
      dtype=object)

In [13]:
# convert x_desc to list of strings
x_desc = [str(i) for i in x_desc]

In [14]:
x_desc[:2]

["['ADP ribosylation factor 5 [Source:HGNC Symbol;Acc:HGNC:658]']",
 "['mannose-6-phosphate receptor, cation dependent [Source:HGNC Symbol;Acc:HGNC:6752]']"]

#### Name emb

In [12]:
transcript_name_emb_path = './BioMedGraphica/embeddings/Entity/transcript_embeddings.npy'
protein_name_emb_path = './BioMedGraphica/embeddings/Entity/protein_embeddings.npy'

transcript_name_emb = np.load(transcript_name_emb_path)
protein_name_emb = np.load(protein_name_emb_path)
name_emb = np.concatenate((transcript_name_emb, protein_name_emb), axis=0)
print(name_emb.shape)
print(transcript_name_emb.shape, protein_name_emb.shape)

(483817, 4)
(278326, 4) (205491, 4)


In [86]:
import os
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from typing import List, Tuple, Dict
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from torch.utils.data import DataLoader, Dataset
from pykeen.training.callbacks import TrainingCallback  # Updated base class for callbacks

from pykeen.models import TransR
from pykeen.training import SLCWATrainingLoop

from torch_geometric.nn import TransformerConv

def prepare_sentences(data: pd.DataFrame) -> Tuple[List[str], List[str]]:
    """
    Convert each row of the data into a formatted sentence and return IDs.

    Args:
        data (pd.DataFrame): DataFrame containing the data.

    Returns:
        Tuple[List[str], List[str]]: List of sentences and list of IDs.
    """
    # Generate the formatted sentences
    sentences = [
        "; ".join(str(value) for key, value in row.items() if key != "BioMedGraphica_ID" and pd.notna(value))
        for row in data.to_dict(orient="records")
    ]
    ids = data[data.columns[0]].tolist()
    return sentences, ids


In [102]:
transcript_language_name_id_df = pd.read_csv('./BioMedGraphica/Entity/Transcript/biomedgraphica_transcript_language_name_id.csv')
protein_language_name_id_df = pd.read_csv('./BioMedGraphica/Entity\Protein/biomedgraphica_protein_language_name_id.csv')

transcript_name_sentence, transcript_ids = prepare_sentences(transcript_language_name_id_df)
protein_name_sentence, protein_ids = prepare_sentences(protein_language_name_id_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_25784\1567783997.py:2: DtypeWarning: Columns (1,2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  protein_language_name_id_df = pd.read_csv('./BioMedGraphica/Entity\Protein/biomedgraphica_protein_language_name_id.csv')


In [ ]:
name_sentence = transcript_name_sentence + protein_name_sentence
name_ids = transcript_ids + protein_ids

# construct the s_name.csv
name_df = pd.DataFrame({'BioMedGraphica_ID': name_ids, 'Name': name_sentence})
name_df.to_csv('./CellTOSG/s_name.csv', index=False)
display(name_df)

,BioMedGraphica_ID,Name
0,BMG_TS000001,ENST00000000233; ENST00000000233.10; ENSG00000...
1,BMG_TS000002,ENST00000000412; ENST00000000412.8; ENSG000000...
2,BMG_TS000003,ENST00000000442; ENST00000000442.11; ENSG00000...
3,BMG_TS000004,ENST00000001008; ENST00000001008.6; ENSG000000...
4,BMG_TS000005,ENST00000001146; ENST00000001146.7; ENSG000000...
...,...,...
483812,BMG_PT205487,UniProt ID:X6RGR3; RefSeq ID:NP_001353609.1
483813,BMG_PT205488,UniProt ID:X6RLR1; RefSeq ID:NP_001268356.1
483814,BMG_PT205489,UniProt ID:X6RLX0; RefSeq ID:XP_054227478.1; R...
483815,BMG_PT205490,UniProt ID:X6RM00; RefSeq ID:XP_047284540.1; R...


#### Desc emb

In [15]:
from typing import List, Tuple, Dict
from torch.utils.data import DataLoader, Dataset

class SentenceDataset(Dataset):
    def __init__(self, sentences: List[str]):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [3]:
import os
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from typing import List, Tuple, Dict
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from torch.utils.data import DataLoader, Dataset
from pykeen.training.callbacks import TrainingCallback  # Updated base class for callbacks

from pykeen.models import TransR
from pykeen.training import SLCWATrainingLoop

c:\Users\hemingzhang\AppData\Local\anaconda3\envs\mkg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class TextEmb():
    def __init__(self, model_path: str = "microsoft/deberta-v3-small", device: str = "cuda"):
        """
        Args:
            model_path (str, optional): Path to the deberta model. Defaults to 'microsoft/deberta-v3-small'.
            device (str, optional): Device to run the model on ('cpu' or 'cuda'). Defaults to 'cpu'.
        """
        self.model_path = model_path
        self.device = device
        self.model = None
        self.tokenizer = None

    def load_model(self):
        """
        Load the deberta model and tokenizer from the specified model path.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path).to(self.device)

    def generate_embeddings(self, sentences: List[str], batch_size: int = 32, text_emb_dim: int = 64) -> List[float]:
        """
        Generate a single-dimensional embedding for each sentence.

        Args:
            sentences (List[str]): List of sentences to embed.
            batch_size (int, optional): Batch size for DataLoader. Defaults to 32.

        Returns:
            List[float]: List of single-dimensional embeddings.
        """
        dataset = SentenceDataset(sentences)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        embeddings = []
        for batch in tqdm(dataloader, desc="Embedding sentences", unit="batch"):
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            batch_embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze().cpu()
            # Adaptive pooling to 64 dimensions (pooling over the hidden dimension)
            projected = torch.nn.functional.adaptive_avg_pool1d(batch_embeddings.unsqueeze(1), output_size=text_emb_dim).squeeze(1)
            embeddings.extend(projected.tolist())
        return embeddings

    def save_embeddings(self, embeddings: List[float], output_npy_path: str) -> None:
        """
        Save embeddings to a .npy file and IDs to a CSV file.

        Args:
            embeddings (List[float]): List of single-dimensional embeddings.
            ids (List[str]): List of corresponding IDs.
            output_npy_path (str): Path to save the .npy file.
            output_csv_path (str): Path to save the index CSV file.
        """
        # Save the embeddings to .npy file
        np.save(output_npy_path, np.array(embeddings))
        print(f"Embeddings saved at {output_npy_path} with shape {np.array(embeddings).shape}")

    def process_data_and_generate_embeddings(self, sentences: List, batch_size: str, text_emb_dim: int) -> Tuple[List[float], List[str]]:

        embeddings = self.generate_embeddings(sentences, batch_size, text_emb_dim)
        # print generated embeddings at shape
        print(f"Generated embeddings at shape {np.array(embeddings).shape}")
        return embeddings

In [ ]:
mapping_table = pd.read_csv('./CellTOSG/brain_sc_output/mapping_table.csv')
display(mapping_table.head())
ids = mapping_table['BioMedGraphica_ID'].tolist()
print(len(ids))
print(ids[:2])

C:\Users\imhem\AppData\Local\Temp\ipykernel_30944\473054218.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  mapping_table = pd.read_csv('./CellTOG/brain_sc_output/mapping_table.csv')


,index,original_id,original_index,BioMedGraphica_ID
0,0,ARF5,ARF5,BMG_TS000001
1,1,M6PR,M6PR,BMG_TS000002
2,2,ESRRA,ESRRA,BMG_TS000003
3,3,FKBP4,FKBP4,BMG_TS000004
4,4,NaN,NaN,BMG_TS000005


483817
['BMG_TS000001', 'BMG_TS000002']


In [ ]:
model_path = "microsoft/deberta-v3-small"
device = "cuda"
batch_size = 128
desc_emb_dim = 1
output_npy_path = "./CellTOSG/x_desc_emb.npy"

In [25]:
encoder = TextEmb(model_path, device)
encoder.load_model()

c:\Users\imhem\anaconda3\envs\mkg\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
embeddings = encoder.process_data_and_generate_embeddings(x_desc, batch_size, desc_emb_dim)

In [ ]:
encoder.save_embeddings(embeddings, output_npy_path)

In [116]:
transcript_desc_df = pd.read_csv('./BioMedGraphica/Entity/Transcript/biomedgraphica_transcript_description.csv')
protein_desc_df = pd.read_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein_description.csv').rename(columns={'UniProt': 'Description'})
desc_df = pd.concat([transcript_desc_df, protein_desc_df], axis=0)
desc_df

,BioMedGraphica_ID,Description
0,BMG_TS000001,ADP ribosylation factor 5 [Source:HGNC Symbol;...
1,BMG_TS000002,"mannose-6-phosphate receptor, cation dependent..."
2,BMG_TS000003,estrogen related receptor alpha [Source:HGNC S...
3,BMG_TS000004,FKBP prolyl isomerase 4 [Source:HGNC Symbol;Ac...
4,BMG_TS000005,cytochrome P450 family 26 subfamily B member 1...
...,...,...
205486,BMG_PT205487,NaN
205487,BMG_PT205488,NaN
205488,BMG_PT205489,NaN
205489,BMG_PT205490,NaN


In [117]:
transcript_df = pd.read_csv('./BioMedGraphica/Entity/Transcript/biomedgraphica_transcript.csv')
protein_df = pd.read_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein.csv')
x_df = pd.concat([transcript_df, protein_df], axis=0)[['BioMedGraphica_ID']]
x_df

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_25784\2600793688.py:2: DtypeWarning: Columns (1,2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  protein_df = pd.read_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein.csv')


,BioMedGraphica_ID
0,BMG_TS000001
1,BMG_TS000002
2,BMG_TS000003
3,BMG_TS000004
4,BMG_TS000005
...,...
205486,BMG_PT205487
205487,BMG_PT205488
205488,BMG_PT205489
205489,BMG_PT205490


In [ ]:
s_desc_df = pd.merge(x_df, desc_df, on='BioMedGraphica_ID', how='left')
s_desc_df = s_desc_df.reset_index(drop=True)[['BioMedGraphica_ID', 'Description']]
display(s_desc_df)
s_desc_df.to_csv('./CellTOSG/s_desc.csv', index=False)

,BioMedGraphica_ID,Description
0,BMG_TS000001,ADP ribosylation factor 5 [Source:HGNC Symbol;...
1,BMG_TS000002,"mannose-6-phosphate receptor, cation dependent..."
2,BMG_TS000003,estrogen related receptor alpha [Source:HGNC S...
3,BMG_TS000004,FKBP prolyl isomerase 4 [Source:HGNC Symbol;Ac...
4,BMG_TS000005,cytochrome P450 family 26 subfamily B member 1...
...,...,...
483812,BMG_PT205487,NaN
483813,BMG_PT205488,NaN
483814,BMG_PT205489,NaN
483815,BMG_PT205490,NaN


#### Seq emb

##### transcript

In [3]:
transcript_seq_path = './BioMedGraphica/Entity/Transcript/biomedgraphica_transcript_biochem.csv'
protein_seq_path = './BioMedGraphica/Entity/Protein/biomedgraphica_protein_biochem.csv'

In [4]:
import pandas as pd
transcript_seq_df = pd.read_csv(transcript_seq_path)
display(transcript_seq_df.head())

,BioMedGraphica_ID,transcript_sequence
0,BMG_TS000001,CUGCUGCUGCUGCGCCCCAUCCCCCCGCGGCCGGCCAGUUCCAGCC...
1,BMG_TS000002,AGAGUGGGGCACAGCGAGGCGCUAGGGGGAACGCUGGCCUCUGAAA...
2,BMG_TS000003,GUCAGCUGGAGGAAGCGGAGUAGGAAGCGGCCGCGAUGUCCUUUUG...
3,BMG_TS000004,CCUACCCCAGCUCUCGCGCCGCGUGCAGAGGUGCUCAAGCCUCCUC...
4,BMG_TS000005,ACAGCCAAUCCCCCGAGCGGCCGCCAACAUGCUCUUUGAGGGCUUG...


In [6]:
transcript_seq_df.shape

(278326, 2)

In [60]:
transcript_sequences = transcript_seq_df['transcript_sequence'].tolist()

In [40]:
import os
import torch
from tqdm import tqdm
from typing import List
from dna_gpt.dna_gpt import DNAGPT
from dna_gpt.tokenizer import KmerTokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [41]:
import os
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from typing import List, Tuple, Dict
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from torch.utils.data import DataLoader, Dataset
from pykeen.training.callbacks import TrainingCallback  # Updated base class for callbacks

from pykeen.models import TransR
from pykeen.training import SLCWATrainingLoop

In [27]:
from typing import List, Tuple, Dict
from torch.utils.data import DataLoader, Dataset

class SentenceDataset(Dataset):
    def __init__(self, sentences: List[str]):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [68]:
class RNAGPT_LM:
    def __init__(self, model_path: str ="", model_name: str = "dna_gpt0.1b_h", device: str = "cpu"):
        """
        RNAGPT-based language model.
        Args:
            model_name (str): Name of the RNAGPT model. Defaults to 'dna_gpt0.1b_h'.
            device (str): Device to run the model on ('cpu' or 'cuda'). Defaults to 'cpu'.
        """
        self.model_path = model_path
        self.model_name = model_name
        self.device = device
        self.model = None
        self.tokenizer = None

    def get_model(self, model_name):
        """
        Initialize the RNAGPT model and tokenizer.
        """
        special_tokens = (
            ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] +
            ["+", '-', '*', '/', '=', "&", "|", "!"] +
            ['M', 'B'] + ['P'] + ['R', 'I', 'K', 'L', 'O', 'Q', 'S', 'U', 'V'] + ['W', 'Y', 'X', 'Z']
        )
        if model_name in ('dna_gpt0.1b_h'):
            tokenizer = KmerTokenizer(6, special_tokens, dynamic_kmer=False)
        else:
            tokenizer = KmerTokenizer(6, special_tokens, dynamic_kmer=True)

        vocab_size = len(tokenizer)
        model = DNAGPT.from_name(model_name, vocab_size)
        return model, tokenizer

    def load_model(self):
        """
        Load the RNAGPT model and tokenizer from the specified model name.
        """
        self.model, self.tokenizer = self.get_model(self.model_name)
        weight_path = os.path.join(self.model_path, f"{self.model_name}.pth")
        self._load_model_weights(weight_path)
        return self

    def _load_model_weights(self, weight_path, dtype=None):
        """
        Load the model weights from a checkpoint file.
        """
        state = torch.load(weight_path, map_location="cpu")
        if 'model' in state.keys():
            self.model.load_state_dict(state['model'], strict=False)
        else:
            self.model.load_state_dict(state, strict=False)
        print(f"loading model weights from {weight_path}")
        self.model.to(device=self.device, dtype=dtype)
        self.model.eval()

    def replace_rna_to_dna(self, sequence):
        # If the input is a list or tuple, extract the first element.
        if isinstance(sequence, (list, tuple)):
            sequence = sequence[0]
        # If the sequence is a float (or any non-string), return 'N'.
        if isinstance(sequence, float):
            return 'N'
        # Otherwise, replace 'U' with 'T' in the string.
        return sequence.replace('U', 'T')
    
    def generate_embeddings(self, sequences: List[str], batch_size: int = 16, max_len: int = 256, text_emb_dim: int = 64) -> np.ndarray:
        """
        Generate tensor embeddings for each RNA sequence. 
        Args:
            sequences (List[str]): List of RNA sequences.
            batch_size (int): Batch size for processing sequences. Defaults to 16.
            max_len (int): Maximum length of the sequences. Defaults to 256.
            text_emb_dim (int): Dimension to which embeddings are pooled. Defaults to 64.
        Returns:
            np.ndarray: A numpy array of shape (num_sequences, text_emb_dim)
        """
        print(f"Generating embeddings for {len(sequences)} sequences.")
        device = self.device
        dataset = SentenceDataset(sequences)
        # Override collate_fn to return the raw list of sequences
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)

        embeddings = []
        for batch in tqdm(dataloader, desc="Embedding sequences", unit="sequence"):
            # Tokenize and prepare input tensors as a list of tensors.
            batch_tokenized = [
                torch.tensor(
                    self.tokenizer.encode(
                        self.replace_rna_to_dna(seq),
                        max_len=max_len,
                        device=device
                    ),
                    dtype=torch.long
                )
                for seq in batch
            ]
            # Pad sequences in the batch to the same length
            input_ids = torch.nn.utils.rnn.pad_sequence(batch_tokenized, batch_first=True, padding_value=0).to(device)
            max_new_tokens = max_len - input_ids.shape[1]

            # Forward pass to compute embeddings
            with torch.no_grad():
                outputs = self.model(input_ids, max_new_tokens)
                # Compute the sequence embedding (mean pooling along token dimension)
                sequence_embedding = torch.mean(outputs, dim=1).squeeze()
                # Adaptive pooling to the desired dimension (pooling over the hidden dimension)
                projected = torch.nn.functional.adaptive_avg_pool1d(sequence_embedding.unsqueeze(1), output_size=text_emb_dim).squeeze(1)
                embeddings.append(projected)

        # Concatenate embeddings from all batches along the first dimension
        embeddings_tensor = torch.cat(embeddings, dim=0)
        print(f"Generated embeddings shape: {embeddings_tensor.shape}")

        # Convert to numpy array and return
        return embeddings_tensor.cpu().numpy()

    def generate_mid_embeddings(self, sequences: List[str], max_len: int = 256) -> torch.Tensor:
        """
        Generate tensor mid embeddings for each RNA sequence. 
        Args:
            sequences (List[str]): List of DNA sequences.
            max_len (int): Maximum length of the sequences. Defaults to 256.
        Returns:
            torch.Tensor: A tensor of shape (num_sequences, embedding_dim) where
                        num_sequences is the number of sequences and
                        embedding_dim is the dimension of the reduced embeddings.
        """
        print(f"Generating embeddings for {len(sequences)} sequences.")
        device = self.device
        embeddings = []

        for sequence in tqdm(sequences, desc="Embedding sequences", unit="sequence"):
            # Tokenize and prepare input tensors
            sequence = self.replace_rna_to_dna(sequence[0])
            input_ids = self.tokenizer.encode(sequence, max_len=max_len, device=device)[None, :]  # Add batch dimension
            input_ids = torch.tensor(input_ids, dtype=torch.long, device=device)
            max_new_tokens = max_len - input_ids.shape[1]

            # Forward pass to compute embeddings
            with torch.no_grad():
                outputs = self.model.generate_mid_embeddings(input_ids, max_new_tokens)
                # Compute the sequence embedding (mean pooling along token dimension)
                sequence_embedding = torch.mean(outputs, dim=1).squeeze()
                embeddings.append(sequence_embedding)

        # Stack embeddings into a single tensor (shape: [num_sequences, embedding_dim])
        embeddings_tensor = torch.stack(embeddings)
        print(f"Generated embeddings shape: {embeddings_tensor.shape}")
        return embeddings_tensor

In [70]:
device = "cuda"
rna_gpt_lm = RNAGPT_LM(model_path=".", model_name="dna_gpt0.1b_h", device=device)
rna_gpt_lm.load_model()

number of parameters: 100.13M
loading model weights from .\dna_gpt0.1b_h.pth


In [19]:
transcript_sequences[0].replace('U', 'T')

'CTGCTGCTGCTGCGCCCCATCCCCCCGCGGCCGGCCAGTTCCAGCCCGCACCCCGCGTCGGTGCCCGCGCCCCTCCCCGGGCCCCGCCATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGCAGATCGAGCGCGCGGCCCGGACCGGGGCGCCGGCCCCGGCGCAGCCCTTCCGCCCCCGCGTCCCTCCAGCCCCGCTCACCTGGGTCTCTGGCCCCGAGTCACCCACCTCATAACGCCCCGGGGCCTCTGCTCTCGGGCGGGTCCCGGTCTGCATCGCCGACCCCGGGGCCTGACACCCGGAGCTGCGGGCCTGGGTGGGGTGAAGCTCCCTCCGCCCCAGCCCGGCTGGTAAGAAGGGAGGATTCCGCCCTTGGAGACGACTTTTAAAACGAGCGCGGCCTACCTCCCGTGCCCCTTGCCTCCCAGTCCTCTCCCGCTCCGCGCCCTCTTTGGAGTTGGCTCACGCCACCCGACTGCCCTCCAGGCCTCAAAGGTAGATAACGACGCGCACCTGGAGGCGCTCCCGCTCTCCGCCCCAGTCACCATCAGCTGTTGTGGCCTCTCCCTTTCCCTGGTCTCTAGGGGGCTCCCTCGCTCCCATCTCCATCCCTGTGCCCCTTTCCGTTGCAGTTGGCTTGGATGCGGCTGGCAAGACCACAATCCTGTACAAACTGAAGTTGGGGGAGATTGTCACCACCATCCCAACCATAGGTGAGCCCGGGGACGAAGCAGGGAGCGGGAGCGCCGCGGCGGGCCCTCGCGGGGTCTGCTCCCAGTTCTGCCACTTTTCTGGAGCTGACCCTGACACCAGATACAGCTATTTGAGAAGTGGGTCAGGGTATCTCTACGTGGATACCGGAGGCAGGGTGGATGTGACCTAGCCCCGCCCTGTTGACCGCCAGTTCTGGGGTTCTGTTCCCCTGGGCCTTGATCATTGCCTTCTGGTTTTGTGTCCCT

In [62]:
len(transcript_sequences)

278326

In [ ]:
# replace RNA to DNA by "sequence.replace('U', 'T')" to the list of sequences
replaced_transcript_sequences = []
for i in transcript_sequences:
    if type(i) == float:
        replaced_transcript_sequences.append('')
    else:
        replaced_transcript_sequences.append(i.replace('U', 'T'))
 
len(replaced_transcript_sequences)

In [74]:
rna_seq = transcript_sequences
batch_size = 64
rna_embedding = rna_gpt_lm.generate_embeddings(sequences=rna_seq, batch_size=batch_size, max_len=256, text_emb_dim=1)

Generating embeddings for 278326 sequences.


Embedding sequences:   0%|          | 0/4349 [00:00<?, ?sequence/s]

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_25784\2961308162.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(
Embedding sequences: 100%|██████████| 4349/4349 [1:01:16<00:00,  1.18sequence/s]

Generated embeddings shape: torch.Size([278326, 1])


In [ ]:
output_npy_path = "./CellTOSG/x_ts_emb.npy"
np.save(output_npy_path, rna_embedding)

##### Merge Protein Sequence to BMG

In [124]:
sequence_df = pd.read_csv('./BioMedGraphica/combined_sequences.csv')
display(sequence_df)

,gene_id,gene_name,contig,start,end,strand,dna_sequence,transcript_sequence,protein_sequence
0,ENSG00000147655,RSPO2,8,107899316.0,108083642.0,-1.0,AGAAGCATCCCAAGCCTCAGCCGGTCTGCATCTCCATCGGAAAGTG...,GGUCUGCAUCUCCAUCGGAAAGUGCGCUUGCCACAUCCCUUCGGAU...,MRQYGECLHSCPSGYYGHRAPDMNRCARCRIENCDSCFSKDFCTKC...
1,ENSG00000147669,POLR2K,8,100150623.0,100154003.0,1.0,ATGTACCAGCGCCGGAAGTTGGTCTCGACACCTGGACTAGCCGGGT...,GGAAGUUGGUCUCGACACCUGGACUAGCCGGGUUGUAUUUGGAAAC...,MDTQKDVQPPKQQPMIYICGECHTENEIKSRDPIRCRECGYRIMYK...
2,ENSG00000147676,MAL2,8,119165034.0,119245673.0,1.0,AGATTCCATGGGCGTAGGACCCTCCGAGCCAGGTGTGGGATATAAT...,GCGCCCGGAGCCCGCGGAGCUGAGCGGCGGCGGCGGCGGCGGCAGG...,MFVSVTAFFFSLLFLGMFLSGMVAQIDANWNFLDFAYHFTVFVFYF...
3,ENSG00000147677,EIF3H,8,116642130.0,116766925.0,-1.0,ACTTAACTGTCAACTCCACGTGGGCGGGGCGTTTTGCACCTCCTGA...,CUCUUUCUUCCUGUCUGCUUGGAAAGAUGGCGUCCCGCAAGGAAGG...,MASRKEGTGSTATSSSSTAGAAGKGKGKGGSGDSAVKQVQIDGLVV...
4,ENSG00000147679,UTP23,8,116766505.0,116849463.0,1.0,TTCATTTCCGGGTGAAACTGGCATTGAGGGTACTGGGGCGTGCGTG...,GGCAUUGAGGGUACUGGGGCGUGCGUGAGGCGUUUACUGAUGCUUC...,MKITRQKHAKKHLGFFRNNFGVREPYQILLDGTFCQAALRGRIQLR...
...,...,...,...,...,...,...,...,...,...
121401,ENSG00000147642,SYBU,8,109573978.0,109691791.0,-1.0,AGCGGCTTCTTCAAGAAACAGATCCTGCAGCATAATCTGCCAGGGC...,AGAGCAAUCGGCGGCAGUGGCGGCGGCGGCGCGUUGGAGCCUCACC...,MGPLRESKKEHRVQHHDKEISRSRIPRLILRPHMPQQQHKVSPASE...
121402,ENSG00000147647,DPYS,8,104330324.0,104467055.0,-1.0,GGATTTGCAGCCTGAGGCAGAGCTCGGGGGCTGTCGGTGGGGACCT...,GGAUUUGCAGCCUGAGGCAGAGCUCGGGGGCUGUCGGUGGGGACCU...,MAAPSRLLIRGGRVVNDDFSEVADVLVEDGVVRALGHDLLPPGGAP...
121403,ENSG00000147649,MTDH,8,97644184.0,97730260.0,1.0,AAGCCGCCATTGTTCCGCCGAGGGAGGACAGCGGGGCCTGGCGCTG...,AAGCCGCCAUUGUUCCGCCGAGGGAGGACAGCGGGGCCUGGCGCUG...,MAARSWQDELAQQAEEGSARLREMLSVGLGFLRTELGLDLGLEPKR...
121404,ENSG00000147650,LRP12,8,104489231.0,104589258.0,-1.0,AGTCTGGCGAGTGGCGGCGCCGGCTAGCCCGAGCGGCGCGAGGCCG...,AGUCUGGCGAGUGGCGGCGCCGGCUAGCCCGAGCGGCGCGAGGCCG...,MACRWSTKESPRWRSALLLLFLAGVYACGETPEQIRAPSGIITSPG...


In [160]:
protein_seq_df = sequence_df[['gene_id', 'gene_name', 'protein_sequence']]
protein_seq_df

,gene_id,gene_name,protein_sequence
0,ENSG00000147655,RSPO2,MRQYGECLHSCPSGYYGHRAPDMNRCARCRIENCDSCFSKDFCTKC...
1,ENSG00000147669,POLR2K,MDTQKDVQPPKQQPMIYICGECHTENEIKSRDPIRCRECGYRIMYK...
2,ENSG00000147676,MAL2,MFVSVTAFFFSLLFLGMFLSGMVAQIDANWNFLDFAYHFTVFVFYF...
3,ENSG00000147677,EIF3H,MASRKEGTGSTATSSSSTAGAAGKGKGKGGSGDSAVKQVQIDGLVV...
4,ENSG00000147679,UTP23,MKITRQKHAKKHLGFFRNNFGVREPYQILLDGTFCQAALRGRIQLR...
...,...,...,...
121401,ENSG00000147642,SYBU,MGPLRESKKEHRVQHHDKEISRSRIPRLILRPHMPQQQHKVSPASE...
121402,ENSG00000147647,DPYS,MAAPSRLLIRGGRVVNDDFSEVADVLVEDGVVRALGHDLLPPGGAP...
121403,ENSG00000147649,MTDH,MAARSWQDELAQQAEEGSARLREMLSVGLGFLRTELGLDLGLEPKR...
121404,ENSG00000147650,LRP12,MACRWSTKESPRWRSALLLLFLAGVYACGETPEQIRAPSGIITSPG...


In [127]:
protein_df = pd.read_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein.csv')
protein_df

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_25784\1568805577.py:1: DtypeWarning: Columns (1,2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  protein_df = pd.read_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein.csv')


,BioMedGraphica_ID,Ensembl_Protein_ID,Ensembl_Protein_ID_Version,Uniprot_ID,RefSeq_ID,NCBI_ID,Protein_Name,HGNC_Symbol
0,BMG_PT000001,ENSP00000000233,ENSP00000000233.5,P84085,NP_001653;NP_001653.1,381,ADP-ribosylation factor 5,ARF5
1,BMG_PT000002,ENSP00000000412,ENSP00000000412.3,P20645,NP_001401249;NP_001401249.1;NP_001401260;NP_00...,4074,Cation-dependent mannose-6-phosphate receptor ...,M6PR
2,BMG_PT000003,ENSP00000000442,ENSP00000000442.6,P11474,NP_001269379.1;NP_001269380;NP_004442;NP_004442.3,2101,Steroid hormone receptor ERR1 (Estrogen recept...,ESRRA
3,BMG_PT000004,ENSP00000001008,ENSP00000001008.4,Q02790,NP_002005;NP_002005.1,2288,Peptidyl-prolyl cis-trans isomerase FKBP4 (PPI...,FKBP4
4,BMG_PT000005,ENSP00000001146,ENSP00000001146.2,Q9NR63,NP_001264671;NP_063938;NP_063938.1,56603,Cytochrome P450 26B1 (EC 1.14.13.-) (Cytochrom...,CYP26B1
...,...,...,...,...,...,...,...,...
205486,BMG_PT205487,NaN,NaN,X6RGR3,NP_001353609.1,NaN,NaN,NaN
205487,BMG_PT205488,NaN,NaN,X6RLR1,NP_001268356.1,NaN,NaN,NaN
205488,BMG_PT205489,NaN,NaN,X6RLX0,XP_054227478.1;XP_016874551.1,NaN,NaN,NaN
205489,BMG_PT205490,NaN,NaN,X6RM00,XP_047284540.1;XP_054227509.1,NaN,NaN,NaN


In [161]:
protein_id_df = protein_df[['BioMedGraphica_ID', 'HGNC_Symbol']]
protein_id_df

,BioMedGraphica_ID,HGNC_Symbol
0,BMG_PT000001,ARF5
1,BMG_PT000002,M6PR
2,BMG_PT000003,ESRRA
3,BMG_PT000004,FKBP4
4,BMG_PT000005,CYP26B1
...,...,...
205486,BMG_PT205487,NaN
205487,BMG_PT205488,NaN
205488,BMG_PT205489,NaN
205489,BMG_PT205490,NaN


In [185]:
protein_seq_dropna_df = protein_seq_df.drop_duplicates(subset='gene_name').dropna().reset_index(drop=True)[['gene_name', 'protein_sequence']]
protein_seq_dropna_df

,gene_name,protein_sequence
0,RSPO2,MRQYGECLHSCPSGYYGHRAPDMNRCARCRIENCDSCFSKDFCTKC...
1,POLR2K,MDTQKDVQPPKQQPMIYICGECHTENEIKSRDPIRCRECGYRIMYK...
2,MAL2,MFVSVTAFFFSLLFLGMFLSGMVAQIDANWNFLDFAYHFTVFVFYF...
3,EIF3H,MASRKEGTGSTATSSSSTAGAAGKGKGKGGSGDSAVKQVQIDGLVV...
4,UTP23,MKITRQKHAKKHLGFFRNNFGVREPYQILLDGTFCQAALRGRIQLR...
...,...,...
17822,SYBU,MGPLRESKKEHRVQHHDKEISRSRIPRLILRPHMPQQQHKVSPASE...
17823,DPYS,MAAPSRLLIRGGRVVNDDFSEVADVLVEDGVVRALGHDLLPPGGAP...
17824,MTDH,MAARSWQDELAQQAEEGSARLREMLSVGLGFLRTELGLDLGLEPKR...
17825,LRP12,MACRWSTKESPRWRSALLLLFLAGVYACGETPEQIRAPSGIITSPG...


In [187]:
bmg_protein_seq_df = pd.merge(protein_id_df, protein_seq_dropna_df, left_on='HGNC_Symbol', right_on='gene_name', how='left')[['BioMedGraphica_ID', 'protein_sequence']]
bmg_protein_seq_df.to_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein_biochem.csv', index=False)

In [5]:
bmg_protein_seq_df = pd.read_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein_biochem.csv')
bmg_protein_seq_df

,BioMedGraphica_ID,protein_sequence
0,BMG_PT000001,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...
1,BMG_PT000002,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...
2,BMG_PT000003,MRTAGPTPAVSWRKRSRKRPRCPFVSYKQPAAPPSEGTRRGGAARP...
3,BMG_PT000004,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...
4,BMG_PT000005,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...
...,...,...
205486,BMG_PT205487,NaN
205487,BMG_PT205488,NaN
205488,BMG_PT205489,NaN
205489,BMG_PT205490,NaN


In [6]:
transcript_seq_df = transcript_seq_df.rename(columns={'transcript_sequence': 'sequence'})
bmg_protein_seq_df = bmg_protein_seq_df.rename(columns={'protein_sequence': 'sequence'})

In [ ]:
s_bio_df = pd.concat([transcript_seq_df, bmg_protein_seq_df], axis=0).reset_index(drop=True)
display(s_bio_df)
s_bio_df.to_csv('./CellTOSG/s_bio.csv', index=False)

,BioMedGraphica_ID,sequence
0,BMG_TS000001,CUGCUGCUGCUGCGCCCCAUCCCCCCGCGGCCGGCCAGUUCCAGCC...
1,BMG_TS000002,AGAGUGGGGCACAGCGAGGCGCUAGGGGGAACGCUGGCCUCUGAAA...
2,BMG_TS000003,GUCAGCUGGAGGAAGCGGAGUAGGAAGCGGCCGCGAUGUCCUUUUG...
3,BMG_TS000004,CCUACCCCAGCUCUCGCGCCGCGUGCAGAGGUGCUCAAGCCUCCUC...
4,BMG_TS000005,ACAGCCAAUCCCCCGAGCGGCCGCCAACAUGCUCUUUGAGGGCUUG...
...,...,...
483812,BMG_PT205487,NaN
483813,BMG_PT205488,NaN
483814,BMG_PT205489,NaN
483815,BMG_PT205490,NaN


##### Protein Emb

In [1]:
from typing import List, Tuple, Dict
from torch.utils.data import DataLoader, Dataset

class SentenceDataset(Dataset):
    def __init__(self, sentences: List[str]):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [6]:
def get_sequence(seq):
    if isinstance(seq, str):
        return seq
    elif isinstance(seq, (list, tuple)) and len(seq) > 0:
        return seq[0]
    else:
        return str(seq)  # Fallback conversion

import os
import torch
import numpy as np
from tqdm import tqdm
from typing import List
from transformers import GPT2LMHeadModel, GPT2Tokenizer

class ProtGPT_LM:
    def __init__(self, model_path: str ="", model_name: str = "prot_gpt2", device: str = "cpu"):
        """
        ProtGPT2-based language model.
        Args:
            model_name (str): Name of the ProtGPT2 model. Defaults to 'prot_gpt2'.
            device (str): Device to run the model on ('cpu' or 'cuda'). Defaults to 'cpu'.
        """
        self.model_path = model_path
        self.model_name = model_name
        self.device = device
        self.model = None
        self.tokenizer = None

    def get_model(self, model_name):
        """
        Initialize the ProtGPT2 model and tokenizer.
        """
        model = GPT2LMHeadModel.from_pretrained(model_name).to(self.device)
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        return model, tokenizer

    def load_model(self):
        """
        Load the ProtGPT2 model and tokenizer from the specified model name.
        """
        self.model, self.tokenizer = self.get_model(self.model_name)
        return self

    def generate_embeddings(self, sequences: List[str], batch_size: int = 16, text_emb_dim: int = 64) -> np.ndarray:
        """
        Generate tensor embeddings for each protein sequence with gradients enabled.
        
        Args:
            sequences (List[str]): List of protein sequences.
            batch_size (int): Batch size for processing sequences. Defaults to 16.
            text_emb_dim (int): Dimension to which embeddings are pooled. Defaults to 64.
        
        Returns:
            np.ndarray: A numpy array of shape (num_sequences, text_emb_dim) where
                        num_sequences is the number of sequences and
                        text_emb_dim is the dimension of the reduced embeddings.
        """
        print(f"Generating embeddings for {len(sequences)} sequences.")
        device = self.device
        self.model.to(device)
        
        # Create a dataset and DataLoader for batching.
        dataset = SentenceDataset(sequences)
        # Use a custom collate_fn so the DataLoader returns the raw list of sequences.
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)
        
        embeddings = []

        for batch in tqdm(dataloader, desc="Embedding sequences", unit="batch"):
            # Tokenize valid sequences in the batch.
            batch_tokenized = [
                torch.tensor(self.tokenizer.encode(get_sequence(seq)), dtype=torch.long)
                for seq in batch if pd.notna(seq)
            ]

            if not batch_tokenized:
                raise ValueError("No valid sequences found in the batch.")

            # Use the tokenizer's pad_token_id if available, otherwise default to 0.
            pad_value = (
                self.tokenizer.pad_token_id
                if hasattr(self.tokenizer, "pad_token_id") and self.tokenizer.pad_token_id is not None
                else 0
            )

            # Pad sequences so that they are all the same length and move the result to the target device.
            input_ids = torch.nn.utils.rnn.pad_sequence(
                batch_tokenized, batch_first=True, padding_value=pad_value
            ).to(device)

            # Get the model's maximum allowed sequence length.
            max_length = self.model.config.n_positions

            # Truncate padded sequences if they exceed the maximum allowed length.
            if input_ids.size(1) > max_length:
                input_ids = input_ids[:, :max_length]
            
            with torch.no_grad():
                outputs = self.model(input_ids, labels=input_ids)
                loss, logits = outputs[:2]
                # Mean pooling over the token dimension => shape: [batch, hidden_dim]
                sequence_embeddings = torch.mean(logits, dim=1)
                # Adaptive pooling to the desired dimension (pooling over the hidden dimension)
                projected = torch.nn.functional.adaptive_avg_pool1d(sequence_embeddings.unsqueeze(1), output_size=text_emb_dim).squeeze(1)
                # Immediately offload to CPU and convert to numpy.
                projected_np = projected.detach().cpu().numpy()
                embeddings.append(projected_np)

        # If embeddings were generated, concatenate them along the first dimension.
        if embeddings:
            embeddings_array = np.concatenate(embeddings, axis=0)
        else:
            embeddings_array = np.empty((0, text_emb_dim))
            
        print(f"Generated embeddings shape: {embeddings_array.shape}")
        
        # Convert to numpy array and return.
        return embeddings_array

In [7]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
device = "cuda"
prot_gpt_lm = ProtGPT_LM(model_name="nferruz/ProtGPT2", device=device)
prot_gpt_lm.load_model()

In [8]:
import pandas as pd
bmg_protein_seq_df = pd.read_csv('./BioMedGraphica/Entity/Protein/biomedgraphica_protein_biochem.csv')
protein_sequences = bmg_protein_seq_df['protein_sequence'].tolist()
len(protein_sequences)

205491

In [ ]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

prot_gpt_lm.model.eval()  # Ensure model is in evaluation mode

with torch.no_grad():
    protein_seq_emb = prot_gpt_lm.generate_embeddings(
        sequences=protein_sequences, 
        batch_size=4, 
        text_emb_dim=1
    )